In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from io import BytesIO
from PIL import Image
from matplotlib import pyplot as plt
import base64
import re
from selenium.webdriver.support.ui import Select
from tqdm import tqdm

In [82]:
import pandas as pd

In [240]:
def get_season_table_data(table):
    
    league_name = table.split('\n')[0]
    table.split('\n')
    team_names = []
    team_in_league = 0
    idx_team_split = {}
    flag = 0
    for i, elem in enumerate(table.split('\n')[1:]):
        if not elem.isdigit():
            if flag == 0:
                team_in_league = int(table.split('\n')[1:][i-2])
                flag = 1
            if not ':' in elem:
                team_names.append(elem)
                idx_team_split[elem] = table.split('\n').index(elem)
    season_results = []

    team_names_idx = {v+1:k for v,k in enumerate(team_names)}
    for team in team_names:
        c = 0
        team_match_results = {'home':[], 'away':[], 'score':[]}
        for i, elem in enumerate(table.split('\n')[idx_team_split[team]+1: idx_team_split[team]+team_in_league]):
            if team_names_idx[i+1] == team:
                c+= 1
            team_match_results['home'].append(team)
            team_match_results['away'].append(team_names_idx[i+c+1])
            team_match_results['score'].append(elem)
        season_results.append(team_match_results)

    '''
    List of home:away:score results each of them is easily transforming to 
    a DataFrame concat using loop for
    '''
    return season_results

In [161]:

tables = {}
screenshots = []
url = 'https://www.championat.com/football/_russiapl/tournament/3859/'
path_to_chrome_driver = '/Users/finlandcowboy/Downloads/chromedriver'
driver = webdriver.Chrome(path_to_chrome_driver)
driver.get(url)
years = driver.find_element(By.NAME, "year")
seasons = years.text.replace(' ', '').split('\n')

for season in tqdm(seasons[10:]):
    
    select = Select(driver.find_element(By.NAME, "year"))
    
    select.select_by_value(season)
    time.sleep(10)
    season_matches_results = driver.find_element(By.LINK_TEXT, "Результаты матчей")
    time.sleep(5)
    season_matches_results.click()

    results_table = driver.find_element(By.XPATH, '//*[@id="scrollable_table_0"]/table')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    base64_screen = results_table.screenshot_as_base64

    tables[season] = results_table.text
    im = Image.open(BytesIO(base64.b64decode(base64_screen)))
    screenshots.append(im)
driver.close()
    

 33%|███▎      | 10/30 [08:29<16:58, 50.95s/it]


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=91.0.4472.114)


In [166]:
driver = webdriver.Chrome(path_to_chrome_driver)
driver.get(url)
years = driver.find_element(By.NAME, "year")
seasons = years.text.replace(' ', '').split('\n')

for season in tqdm(seasons[27:]):
    select = Select(driver.find_element(By.NAME, "year"))
    
    select.select_by_value(season)
    season_matches_results = driver.find_element(By.LINK_TEXT, "Результаты матчей")
    season_matches_results.click()

    results_table = driver.find_element(By.XPATH, '//*[@id="scrollable_table_0"]/table')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    base64_screen = results_table.screenshot_as_base64
    tables[season] = results_table.text
    im = Image.open(BytesIO(base64.b64decode(base64_screen)))
    screenshots.append(im)
driver.close()

100%|██████████| 3/3 [00:23<00:00,  7.89s/it]


In [167]:
tables.keys()

dict_keys(['2021/2022', '2020/2021', '2019/2020', '2018/2019', '2017/2018', '2016/2017', '2015/2016', '2014/2015', '2013/2014', '2012/2013', '2011/2012', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996', '1995', '1994', '1993', '1992'])

In [241]:
data = get_season_table_data(tables['2020/2021'])[0]
df = pd.DataFrame(data)

In [242]:
dfs = []
for key in tables.keys():
    if key == '2021/2022':
        continue
    data = get_season_table_data(tables[key])
    for d in data:
        df = pd.DataFrame(d)
        df['year'] = key
        dfs.append(df)
data = pd.concat(list(map(pd.DataFrame, dfs)))

In [253]:
def more_less(goals, more=True):
    if more:
        
    else:
        

,home,away,score,year
0,ЦСКА,Зенит,0 : 4,2019/2020
1,ЦСКА,Локомотив М,1 : 0,2019/2020
2,ЦСКА,Краснодар,3 : 2,2019/2020
3,ЦСКА,Ростов,1 : 3,2019/2020
4,ЦСКА,Динамо М,0 : 1,2019/2020
5,ЦСКА,Спартак М,2 : 0,2019/2020
6,ЦСКА,Арсенал,0 : 1,2019/2020
7,ЦСКА,Уфа,0 : 0,2019/2020
8,ЦСКА,Рубин,1 : 1,2019/2020
9,ЦСКА,Урал,1 : 1,2019/2020


In [254]:
data.to_csv('russian_premier_league_scores.csv', index=False)

### List of things

team_names_idx - Approved 
\
team_match_result

In [120]:
league_name = tables[2010].split('\n')[0]
tables[2010].split('\n')
team_names = []
team_in_league = 0
idx_team_split = {}
flag = 0
for i, elem in enumerate(tables[2010].split('\n')[1:]):
    if not elem.isdigit():
        if flag == 0:
            team_in_league = int(tables[2010].split('\n')[1:][i-2])
            flag = 1
        if not ':' in elem:
            team_names.append(elem)
            idx_team_split[elem] = tables[2010].split('\n').index(elem)

season_results = []
            
team_names_idx = {v+1:k for v,k in enumerate(team_names)}

for team in team_names:
    team_match_results = {'home':[], 'away':[], 'score':[]}
    
    c = 0
    for i, elem in enumerate(tables[2010].split('\n')[idx_team_split[team]+1: idx_team_split[team]+16]):
        team_match_results['home'].append(team)
        team_match_results['away'].append(team_names_idx[i+1])
        team_match_results['score'].append(elem)
    season_results.append(team_match_results)
